In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import torch
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet101
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import lr_scheduler
from database2 import DehazingDataset
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install opencv-contrib-python

In [ ]:
# def estimate_transmission(image, window_size=15):
#     # Convert the image to float32 format
#     image = image.astype(np.float32) / 255.0

#     # Compute the dark channel of the image
#     dark_channel = np.min(image, axis=2)

#     # Compute the minimum value in the dark channel using a local window
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (window_size, window_size))
#     min_dark_channel = cv2.erode(dark_channel, kernel)

#     # Estimate the atmospheric light
#     atmospheric_light = np.percentile(image, 99, axis=(0, 1))

#     # Compute the transmission map
#     transmission = 1 - 0.95 * (dark_channel / min_dark_channel)

#     # Clip transmission values to ensure they are within [0, 1]
#     transmission = np.clip(transmission, 0, 1)

#     return transmission, atmospheric_light

def estimate_transmission_map(hazy_image, window_size=15, omega=0.95):
    # Convert hazy image to numpy array
    hazy_np = np.array(hazy_image)
    
    # Compute dark channel of hazy image
    dark_channel = np.min(hazy_np, axis=2)
    
    # Estimate atmospheric light as the maximum intensity in the dark channel
    atmospheric_light = np.max(dark_channel)
    
    # Normalize dark channel
    normalized_dark_channel = dark_channel / atmospheric_light
    
    # Compute transmission map using the atmospheric light and omega parameter
    transmission_map = 1 - omega * normalized_dark_channel
    
    # Apply guided filter for refinement (optional but recommended)
    # You may need to install the 'cv2' library for this step
    
    transmission_map = cv2.ximgproc.guidedFilter(hazy_np.astype(np.float32), transmission_map.astype(np.float32), radius=window_size, eps=1e-3)
    
    return transmission_map, -1

def generate_transmission_maps(hazy_images, window_size=15):
    transmission_maps = []
    atmospheric_lights = []

    for hazy_image in hazy_images:
        
        transmission, atmospheric_light = estimate_transmission_map(hazy_image, window_size)
        transmission_maps.append(transmission)
        atmospheric_lights.append(atmospheric_light)

    return transmission_maps, atmospheric_lights

def visualize_transmission_maps(transmission_maps):
    num_images = len(transmission_maps)
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for i, transmission_map in enumerate(transmission_maps):
        axes[i].imshow(transmission_map, cmap='gray')
        axes[i].axis('off')
        axes[i].set_title(f"Transmission Map {i+1}")
    plt.show()




# Load Dataset

In [ ]:
root_dir = '../Task2Dataset'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

# DIFFERENT TRANSFORM [USED BY DEEPLAB]
# transform = transforms.Compose([
#     transforms.Resize((512, 512)),  # Resize input image to match model's input size
#     transforms.ToTensor(),           # Convert PIL image to tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
# ])

transform = transforms.Compose([
    transforms.ToTensor(),   
    transforms.Normalize(mean = [0.5, 0.5, 0.5],std = [0.5, 0.5, 0.5])
])

train_dataset = DehazingDataset(train_dir, transform)
val_dataset = DehazingDataset(val_dir, transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
def visualize_hazy_transmission_pairs(hazy_images, transmission_maps):
    num_images = len(hazy_images)
    num_pairs = num_images // 4
    num_rows = num_pairs * 2  # Each pair will occupy two rows

    fig, axes = plt.subplots(num_rows, 4, figsize=(20, 4*num_rows))
    
    for i in range(num_pairs):
        for j in range(4):
            hazy_idx = i * 4 + j
            map_idx = i * 4 + j
            if hazy_idx < num_images:
                axes[i*2, j].imshow(hazy_images[hazy_idx])
                axes[i*2, j].set_title(f"Hazy Image {hazy_idx+1}")
                axes[i*2, j].axis('off')
                
                if map_idx < len(transmission_maps):
                    axes[i*2+1, j].imshow(transmission_maps[map_idx], cmap='gray')
                    axes[i*2+1, j].set_title(f"Transmission Map {map_idx+1}")
                    axes[i*2+1, j].axis('off')
                
    plt.tight_layout()
    plt.show()


In [ ]:
def denormalize(image):
    """Denormalize a tensor image."""
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    image = image.numpy().transpose((1, 2, 0))
    image = std * image + mean
    image = np.clip(image, 0, 1)
    return image
    
hazy_images_list = []
transmission_maps_list = []

num_images = 0
for hazy_imgs, clean_imgs in train_dataloader:
    # CREATE TRANSMISSION MAPS
    hazy_imgs = hazy_imgs[:5]  # Assuming you want to process the first 5 hazy images only

    denormalized_imgs = [denormalize(img) for img in hazy_imgs]

    transmission_maps, _ = generate_transmission_maps(denormalized_imgs)

    # VISUALIZE TRANSMISSION MAPS IN ROWS OF 5
    transmission_maps_list.extend(transmission_maps)
    hazy_images_list.extend(denormalized_imgs)


    num_images += 5

    if num_images >= 200:
        break

visualize_hazy_transmission_pairs(hazy_images_list, transmission_maps_list)

In [ ]:
# Load the pre-trained semantic segmentation model
model = deeplabv3_resnet101(pretrained=True)
model.eval()

# Define transformations for pre-processing
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Resize input image to match model's input size
    transforms.ToTensor(),           # Convert PIL image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
])

# Load and pre-process the hazy image
import time
for i in range(500):
    hazy_image_path = hazy_images_paths[i]
    hazy_image = Image.open(hazy_image_path)
    hazy_image = transform(hazy_image).unsqueeze(0)  # Add batch dimension

    # Perform semantic segmentation inference
    with torch.no_grad():
        output = model(hazy_image)['out'][0]

    # Convert output tensor to numpy array
    semantic_map = torch.argmax(output, dim=0).numpy()

    # Visualize the segmented semantic map
    plt.imshow(semantic_map)
    plt.colorbar()
    plt.show()


# View images from dataloader in rows

In [ ]:
model = deeplabv3_resnet101(pretrained=True)
model.eval()

num_images = 200
count = 0
fig, axes = plt.subplots(num_images // 5, 5, figsize=(20, 4*num_images // 5))

# for hazy images
for images, clean_imgs in train_dataloader:
    # Perform semantic segmentation inference
    with torch.no_grad():
        outputs = model(images)['out']
        segmentation_maps = torch.argmax(outputs, dim=1).numpy()

    # Visualize segmentation maps in rows of 5 images
    for i in range(images.size(0)):
        ax = axes[count // 5, count % 5]
        ax.imshow(segmentation_maps[i], cmap='viridis')
        ax.axis('off')
        ax.set_title(f'Image {count+1}')
        count += 1
        if count >= num_images:
            break
    if count >= num_images:
        break

plt.tight_layout()
plt.show()